# Obtain data and configure access
Data should be downloaded from https://zenodo.org/record/1161203

In [ ]:
import os
# get initial path and data path, verify data path
notebook_path = os.getcwd()

data_path = os.path.expanduser('~/data/uci_data')
assert os.path.isdir(data_path)

best_state_dict_file = os.path.join(data_path, 'best_state_dict.pt')

# Options and hyperparameters

In [ ]:

train_batch_size = 64

val_batch_size = 512

learning_rate = 3e-4

num_training_steps = int(2e5)

anneal_learning_rate = True

grad_norm_clip_value = 5.0

#'affine-coupling', 'quadratic-coupling', 'rq-coupling', 'affine-autoregressive'
#'quadratic-autoregressive', 'rq-autoregressive'
base_transform_type = 'rq-autoregressive' 

# 'permutation', 'lu', 'svd'
linear_transform_type = 'lu'  

num_flow_steps = 10

# Number of hidden features to use in coupling/autoregressive nets
hidden_features = 256  

tail_bound = 3  # Box is on [-bound, bound]^2

num_bins = 8  # Number of bins to use for piecewise transforms.

num_transform_blocks = 2  # Number of blocks to use in coupling/autoregressive nets

use_batch_norm = False

dropout_probability = 0.25  # Dropout probability for coupling/autoregressive nets

monitor_interval = 250  # Interval in steps at which to report training stats.

seed = 1638128  # Random seed for PyTorch and NumPy

dtype = 'float32'  # data will be cast to this

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
pl.plot([1,2,3])

In [ ]:
#import outside packages
import argparse
import json
import numpy as np
import torch
import os

from time import sleep

from torch import optim
from torch.nn import functional as F
from torch.nn.utils import clip_grad_norm_
from torch.utils import data

from tqdm import tqdm

In [ ]:
#import from pyknos
from pyknos.nflows import distributions, transforms, flows

## options and hyperparameters

In [ ]:
train_batch_size = 64

num_training_steps = int(2e5)

anneal_learning_rate = True

num_flow_steps = 10

#'affine-coupling', 'quadratic-coupling', 'rq-coupling', 'affine-autoregressive'
#'quadratic-autoregressive', 'rq-autoregressive'
base_transform_type = 'rq-autoregressive' 

# 'permutation', 'lu', 'svd'
linear_transform_type = 'lu'

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type('torch.cpu.FloatTensor')

## Load miniboone data

In [ ]:
#based on nsf/data/miniboone.py
dataset_name = 'miniboone'

#load data
datafile = os.path.join(data_path, 'miniboone', 'data.npy')
assert os.path.exists(datafile)
data_all = np.load(datafile).astype(dtype)

#divide into train, test, val sets
N_test = int(0.1 * data_all.shape[0])
data_test = data_all[-N_test:]
tmp = data_all[0:-N_test]
N_validate = int(0.1 * tmp.shape[0])
data_validate = tmp[-N_validate:]
data_train = tmp[0:-N_validate]

#normalize
tmp = np.vstack((data_train, data_validate))
mu = tmp.mean(axis=0)
s = tmp.std(axis=0)
data_train = (data_train - mu) / s
data_val = (data_validate - mu) / s
data_test = (data_test - mu) / s

#create data loaders
train_loader = data.DataLoader(dataset=data_train, batch_size=train_batch_size, shuffle=True, drop_last=True)
val_loader = data.DataLoader(dataset=data_val, batch_size=train_batch_size, shuffle=True, drop_last=True)
test_loader = data.DataLoader(dataset=data_test, batch_size=train_batch_size, shuffle=False, drop_last=False)

def batch_generator(loader, num_batches=int(1e10)):
    batch_counter = 0
    while True:
        for batch in loader:
            yield batch
            batch_counter += 1
            if batch_counter == num_batches:
                return
train_generator = batch_generator(train_loader)
test_batch = next(iter(train_loader)).to(device)

assert data_train.ndim == 2
features = data_train.shape[-1]

In [ ]:
# define convenience functions for creating the flow transform based on the above settings
def create_linear_transform():
    if linear_transform_type == 'permutation':
        return transforms.RandomPermutation(features=features)
    elif linear_transform_type == 'lu':
        return transforms.CompositeTransform([
            transforms.RandomPermutation(features=features),
            transforms.LULinear(features, identity_init=True)
        ])
    elif linear_transform_type == 'svd':
        return transforms.CompositeTransform([
            transforms.RandomPermutation(features=features),
            transforms.SVDLinear(features, num_householder=10, identity_init=True)
        ])
    else:
        raise ValueError


def create_base_transform(i):
    if base_transform_type == 'affine-coupling':
        return transforms.AffineCouplingTransform(
            mask=utils.create_alternating_binary_mask(features, even=(i % 2 == 0)),
            transform_net_create_fn=lambda in_features, out_features: nn_.ResidualNet(
                in_features=in_features,
                out_features=out_features,
                hidden_features=hidden_features,
                context_features=None,
                num_blocks=num_transform_blocks,
                activation=F.relu,
                dropout_probability=dropout_probability,
                use_batch_norm=use_batch_norm
            )
        )
    elif base_transform_type == 'quadratic-coupling':
        return transforms.PiecewiseQuadraticCouplingTransform(
            mask=utils.create_alternating_binary_mask(features, even=(i % 2 == 0)),
            transform_net_create_fn=lambda in_features, out_features: nn_.ResidualNet(
                in_features=in_features,
                out_features=out_features,
                hidden_features=hidden_features,
                context_features=None,
                num_blocks=num_transform_blocks,
                activation=F.relu,
                dropout_probability=dropout_probability,
                use_batch_norm=use_batch_norm
            ),
            num_bins=num_bins,
            tails='linear',
            tail_bound=tail_bound,
            apply_unconditional_transform=apply_unconditional_transform
        )
    elif base_transform_type == 'rq-coupling':
        return transforms.PiecewiseRationalQuadraticCouplingTransform(
            mask=utils.create_alternating_binary_mask(features, even=(i % 2 == 0)),
            transform_net_create_fn=lambda in_features, out_features: nn_.ResidualNet(
                in_features=in_features,
                out_features=out_features,
                hidden_features=hidden_features,
                context_features=None,
                num_blocks=num_transform_blocks,
                activation=F.relu,
                dropout_probability=dropout_probability,
                use_batch_norm=use_batch_norm
            ),
            num_bins=num_bins,
            tails='linear',
            tail_bound=tail_bound,
            apply_unconditional_transform=apply_unconditional_transform
        )
    elif base_transform_type == 'affine-autoregressive':
        return transforms.MaskedAffineAutoregressiveTransform(
            features=features,
            hidden_features=hidden_features,
            context_features=None,
            num_blocks=num_transform_blocks,
            use_residual_blocks=True,
            random_mask=False,
            activation=F.relu,
            dropout_probability=dropout_probability,
            use_batch_norm=use_batch_norm
        )
    elif base_transform_type == 'quadratic-autoregressive':
        return transforms.MaskedPiecewiseQuadraticAutoregressiveTransform(
            features=features,
            hidden_features=hidden_features,
            context_features=None,
            num_bins=num_bins,
            tails='linear',
            tail_bound=tail_bound,
            num_blocks=num_transform_blocks,
            use_residual_blocks=True,
            random_mask=False,
            activation=F.relu,
            dropout_probability=dropout_probability,
            use_batch_norm=use_batch_norm
        )
    elif base_transform_type == 'rq-autoregressive':
        return transforms.MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
            features=features,
            hidden_features=hidden_features,
            context_features=None,
            num_bins=num_bins,
            tails='linear',
            tail_bound=tail_bound,
            num_blocks=num_transform_blocks,
            use_residual_blocks=True,
            random_mask=False,
            activation=F.relu,
            dropout_probability=dropout_probability,
            use_batch_norm=use_batch_norm
        )
    else:
        raise ValueError


def create_transform():
    transform = transforms.CompositeTransform([
        transforms.CompositeTransform([
            create_linear_transform(),
            create_base_transform(i)
        ]) for i in range(num_flow_steps)
    ] + [
        create_linear_transform()
    ])
    return transform

In [ ]:
# create the flow
distribution = distributions.StandardNormal((features,))
transform = create_transform()
flow = flows.Flow(transform, distribution).to(device)

n_params = utils.get_num_parameters(flow)
print('There are {} trainable parameters in this model.'.format(n_params))

In [ ]:
# create optimizer
optimizer = optim.Adam(flow.parameters(), lr=learning_rate)
if anneal_learning_rate:
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, num_training_steps, 0)
else:
    scheduler = None

## Train

In [ ]:
tbar = tqdm(range(num_training_steps))
best_val_score = -1e10

#loss_vals = np.full(num_training_steps, np.nan)

val_densities = []

for step in tbar:
    
    flow.train()  # set flow into training mode
    
    if anneal_learning_rate:
        scheduler.step(step)
    optimizer.zero_grad()

    batch = next(train_generator).to(device)
    log_density = flow.log_prob(batch)
    loss = - torch.mean(log_density)
    loss.backward()  # calculate gradients
    
    if grad_norm_clip_value is not None:
        clip_grad_norm_(flow.parameters(), grad_norm_clip_value)
        
    optimizer.step()

    #writer.add_scalar(tag='loss', scalar_value=loss.item(), global_step=step)

    if (step + 1) % monitor_interval == 0:
        flow.eval()

        with torch.no_grad():
            # compute validation score
            running_val_log_density = 0
            for val_batch in val_loader:
                log_density_val = flow.log_prob(val_batch.to(device).detach())
                mean_log_density_val = torch.mean(log_density_val).detach()
                running_val_log_density += mean_log_density_val
            running_val_log_density /= len(val_loader)

        val_densities.append(running_val_log_density.item())
            
        if running_val_log_density > best_val_score:
            best_val_score = running_val_log_density
            #path = os.path.join(cutils.get_checkpoint_root(), '{}-best-val-{}.t'.format(dataset_name, timestamp))
            torch.save(flow.state_dict(), best_state_dict_file)

        # compute reconstruction
        with torch.no_grad():
            test_batch_noise = flow.transform_to_noise(test_batch)
            test_batch_reconstructed, _ = flow._transform.inverse(test_batch_noise)
        errors = test_batch - test_batch_reconstructed
        max_abs_relative_error = torch.abs(errors / test_batch).max()
        average_abs_relative_error = torch.abs(errors / test_batch).mean()
        #writer.add_scalar('max-abs-relative-error', max_abs_relative_error, global_step=step)
        #writer.add_scalar('average-abs-relative-error', average_abs_relative_error, global_step=step)

        summaries = {
            'val': running_val_log_density.item(),
            'best-val': best_val_score.item(),
            'max-abs-relative-error': max_abs_relative_error.item(),
            'average-abs-relative-error': average_abs_relative_error.item()
        }
        #for summary, value in summaries.items():
        #    writer.add_scalar(tag=summary, scalar_value=value, global_step=step)

In [ ]:
# load best val model
#path = os.path.join(cutils.get_checkpoint_root(),
#                    '{}-best-val-{}.t'.format(dataset_name, timestamp))
flow.load_state_dict(torch.load(best_state_dict_file))
flow.eval()  # put flow in evaluation mode

# calculate log-likelihood on test set
with torch.no_grad():
    log_likelihood = torch.Tensor([])
    for batch in tqdm(test_loader):
        log_density = flow.log_prob(batch.to(device))
        log_likelihood = torch.cat([
            log_likelihood,
            log_density
        ])
#path = os.path.join(log_dir, '{}-{}-log-likelihood.npy'.format(
#    dataset_name,
#    base_transform_type
#))
#np.save(path, utils.tensor2numpy(log_likelihood))
mean_log_likelihood = log_likelihood.mean()
std_log_likelihood = log_likelihood.std()

# save log-likelihood
s = 'Final score for {}: {:.2f} +- {:.2f}'.format(
    dataset_name.capitalize(),
    mean_log_likelihood.item(),
    2 * std_log_likelihood.item() / np.sqrt(len(data_test))
)
print(s)